In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
df = pd.read_csv("/Users/michaelhein/Programs/Dicebot/outputs/output.txt")

In [4]:
df.head()

,totalDice,player,numPlayerDice,quantityDeclared,diceNumDeclared,1s,2s,3s,4s,5s,6s
0,25,Agg-High,5,9,4,2,0,0,3,0,0
1,24,Agg-Low,5,8,4,0,0,1,2,2,0
2,24,Cond-Low,5,9,2,1,2,1,0,0,1
3,23,Cond-High,5,9,3,1,0,3,0,1,0
4,22,Conserv,4,1,4,0,1,0,2,0,1


In [5]:
df["totalDice"]

0         25
1         24
2         24
3         23
4         22
          ..
590880     6
590881     6
590882     5
590883     4
590884     4
Name: totalDice, Length: 590885, dtype: int64

In [6]:
df.shape

(590885, 11)

In [7]:
df[["totalDice", "player", "numPlayerDice", "quantityDeclared", "diceNumDeclared"]]

,totalDice,player,numPlayerDice,quantityDeclared,diceNumDeclared
0,25,Agg-High,5,9,4
1,24,Agg-Low,5,8,4
2,24,Cond-Low,5,9,2
3,23,Cond-High,5,9,3
4,22,Conserv,4,1,4
...,...,...,...,...,...
590880,6,Agg-High,1,2,5
590881,6,Agg-Low,1,3,4
590882,5,Cond-High,3,2,3
590883,4,Agg-Low,1,1,4


In [8]:
features = df[df.columns[:5]]
features.head()

,totalDice,player,numPlayerDice,quantityDeclared,diceNumDeclared
0,25,Agg-High,5,9,4
1,24,Agg-Low,5,8,4
2,24,Cond-Low,5,9,2
3,23,Cond-High,5,9,3
4,22,Conserv,4,1,4


In [9]:
targets = df[df.columns[5:]]
targets.head()

,1s,2s,3s,4s,5s,6s
0,2,0,0,3,0,0
1,0,0,1,2,2,0
2,1,2,1,0,0,1
3,1,0,3,0,1,0
4,0,1,0,2,0,1


In [10]:
CondHigh = df[df["player"] == "Cond-High"]
CondHigh = CondHigh.drop(columns = ["player"])
CondHigh.head()

,totalDice,numPlayerDice,quantityDeclared,diceNumDeclared,1s,2s,3s,4s,5s,6s
3,23,5,9,3,1,0,3,0,1,0
8,22,5,8,4,0,1,1,2,0,1
13,20,5,8,3,0,1,3,1,0,0
18,19,5,8,1,4,0,1,0,0,0
23,17,4,6,1,2,0,0,0,1,1


In [11]:
td, test = train_test_split(CondHigh, test_size=0.1)

In [12]:
train, develop = train_test_split(CondHigh, test_size=0.2)

In [13]:
train.shape

(90384, 10)

In [14]:
develop.shape

(22596, 10)

In [15]:
test.shape

(11298, 10)

In [16]:
x_train = train[train.columns[:4]]
y_train = train[train.columns[4:]]

x_develop = develop[develop.columns[:4]]
y_develop = develop[develop.columns[4:]]

x_test = test[test.columns[:4]]
y_test = test[test.columns[4:]]

In [17]:
x_train_tensor = torch.from_numpy(x_train.to_numpy()).float()
y_train_tensor = torch.from_numpy(y_train.to_numpy()).float()

x_develop_tensor = torch.from_numpy(x_develop.to_numpy()).float()
y_develop_tensor = torch.from_numpy(y_develop.to_numpy()).float()

x_test_tensor = torch.from_numpy(x_test.to_numpy()).float()
y_test_tensor = torch.from_numpy(y_test.to_numpy()).float()

In [18]:
train_dataset = list(zip(x_train_tensor, y_train_tensor))
develop_dataset = list(zip(x_develop_tensor, y_develop_tensor))
test_dataset = list(zip(x_test_tensor, y_test_tensor))

In [19]:
class Net(nn.Module):
    def __init__(self, num_layers, layers_size):
        super(Net, self).__init__()
        self.hidden_layers = nn.ModuleList([nn.Linear(4, layers_size)])
        self.hidden_layers.extend([nn.Linear(layers_size, layers_size) for i in range(1, num_layers-1)])
        self.output = nn.Linear(layers_size, 6)
        self.activation = nn.ReLU()
        
    def forward(self, x):
        z = x
        for layer in self.hidden_layers:
            z = self.activation(layer(z))
        return self.output(z)

In [20]:
class Trainer():
    def __init__(self, net, optim, loss_function, train_loader):
        self.net = net
        self.optim = optim
        self.loss_function = loss_function
        self.train_loader = train_loader
        
    def train(self, epochs, verbose=True):
        losses = []
        for epoch in range(epochs):
            epoch_loss = 0.0
            epoch_steps = 0
            for data in self.train_loader:
                x = data[0]
                y = data[1]
                
                self.optim.zero_grad()
                
                preds = self.net.forward(x)
                
                loss = self.loss_function(preds, y)
                
                loss.backward()
                
                self.optim.step()
                
                epoch_loss += loss.item()
                epoch_steps += 1
                
            losses.append(epoch_loss / epoch_steps)
            if verbose: print("epoch [%d]: loss %.3f" % (epoch+1, losses[-1]))
        return losses

In [21]:
## Hyperparamters

num_epochs = 30
batch_size = 256
learning_rate = 0.02

In [22]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
develop_loader = DataLoader(dataset=develop_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [23]:
net = Net(4, 16)
opt = optim.Adam(net.parameters(), lr=learning_rate)
loss_function = nn.MSELoss()

trainer = Trainer(net=net, optim=opt, loss_function=loss_function, train_loader=train_loader)

losses = trainer.train(num_epochs)

epoch [1]: loss 0.332
epoch [2]: loss 0.280
epoch [3]: loss 0.248
epoch [4]: loss 0.242
epoch [5]: loss 0.239
epoch [6]: loss 0.241
epoch [7]: loss 0.235
epoch [8]: loss 0.236
epoch [9]: loss 0.234
epoch [10]: loss 0.233
epoch [11]: loss 0.230
epoch [12]: loss 0.229
epoch [13]: loss 0.232
epoch [14]: loss 0.234
epoch [15]: loss 0.226
epoch [16]: loss 0.227
epoch [17]: loss 0.229
epoch [18]: loss 0.228
epoch [19]: loss 0.248
epoch [20]: loss 0.238
epoch [21]: loss 0.244
epoch [22]: loss 0.235
epoch [23]: loss 0.234
epoch [24]: loss 0.239
epoch [25]: loss 0.235
epoch [26]: loss 0.239
epoch [27]: loss 0.237
epoch [28]: loss 0.239
epoch [29]: loss 0.238
epoch [30]: loss 0.236


In [24]:
CondHigh.head()

,totalDice,numPlayerDice,quantityDeclared,diceNumDeclared,1s,2s,3s,4s,5s,6s
3,23,5,9,3,1,0,3,0,1,0
8,22,5,8,4,0,1,1,2,0,1
13,20,5,8,3,0,1,3,1,0,0
18,19,5,8,1,4,0,1,0,0,0
23,17,4,6,1,2,0,0,0,1,1


In [25]:
train_dataset[0]

(tensor([9., 1., 4., 2.]), tensor([0., 1., 0., 0., 0., 0.]))

In [26]:
train_dataset[1]

(tensor([21.,  5.,  8.,  1.]), tensor([2., 0., 1., 0., 1., 1.]))

In [27]:
x_train.head()

,totalDice,numPlayerDice,quantityDeclared,diceNumDeclared
158190,9,1,4,2
423753,21,5,8,1
229494,14,2,3,5
139262,7,3,3,2
381543,3,1,2,2


In [28]:
net.forward(train_dataset[0][0])

tensor([0.0516, 0.8895, 0.0284, 0.0228, 0.0329, 0.1014],
       grad_fn=<AddBackward0>)

In [29]:
net.forward(train_dataset[1][0])

tensor([2.2379, 0.6938, 0.5566, 0.5449, 0.5921, 0.7920],
       grad_fn=<AddBackward0>)

In [30]:
develop_dataset[0]

(tensor([11.,  1.,  4.,  3.]), tensor([0., 0., 1., 0., 0., 0.]))

In [31]:
develop_dataset[1]

(tensor([18.,  4.,  6.,  3.]), tensor([1., 0., 2., 1., 0., 0.]))

In [32]:
develop_dataset[2]

(tensor([5., 2., 1., 5.]), tensor([1., 0., 0., 0., 1., 0.]))

In [33]:
develop_dataset[3]

(tensor([23.,  5.,  9.,  3.]), tensor([1., 1., 3., 0., 0., 0.]))

In [34]:
net.forward(develop_dataset[0][0])

tensor([ 0.0239, -0.0010,  0.8289, -0.0109,  0.0208,  0.1188],
       grad_fn=<AddBackward0>)

In [35]:
net.forward(develop_dataset[1][0])

tensor([0.4974, 0.4216, 1.6331, 0.4222, 0.4480, 0.7424],
       grad_fn=<AddBackward0>)

In [36]:
net.forward(develop_dataset[2][0])

tensor([0.1980, 0.2585, 0.1140, 0.1844, 1.0836, 0.3157],
       grad_fn=<AddBackward0>)

In [37]:
net.forward(develop_dataset[3][0])

tensor([0.6167, 0.4122, 2.4518, 0.5147, 0.5540, 0.6449],
       grad_fn=<AddBackward0>)

In [38]:
torch.save(net.state_dict(), "/Users/michaelhein/Programs/Dicebot/models/conditional_high_lvl_stat.obj")